In [ ]:
import torch
import torch.fft
import numpy as np
import sys
# Add your custom path
gems_tco_path = "/Users/joonwonlee/Documents/GEMS_TCO-1/src"
sys.path.append(gems_tco_path)
import os
import logging
import argparse # Argument parsing

# Data manipulation and analysis
import pandas as pd
import numpy as np
import pickle
import torch
import torch.optim as optim
import copy                    # clone tensor
import time
from sklearn.neighbors import BallTree
# Custom imports


from GEMS_TCO import kernels_reparam_space_time_gpu as kernels_reparam_space_time_gpu

from GEMS_TCO import kernels_reparam_space_time_gpu_past1 as kernels_reparam_space_time_gpu_past1

from GEMS_TCO import kernels_gpu_st_simulation_column as kernels_gpu_st_simulation_column
# from GEMS_TCO import kernels_gpu_past1  as kernels_gpu_past1

from GEMS_TCO import orderings as _orderings 
from GEMS_TCO import alg_optimization, alg_opt_Encoder

from typing import Optional, List, Tuple
from pathlib import Path
import typer
import json
from json import JSONEncoder
from GEMS_TCO import configuration as config
from GEMS_TCO.data_loader import load_data2, exact_location_filter
from GEMS_TCO import debiased_whittle
from torch.nn import Parameter

# --- 1. CONFIGURATION ---
# Check for Mac GPU (MPS) first, then CUDA, then CPU
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Optional: Force CPU if you encounter Cholesky errors later
# DEVICE = torch.device("cpu") 
DTYPE = torch.float32 if DEVICE.type == 'mps' else torch.float64

print(f"Simulating on: {DEVICE}")

# TRUE PARAMETERS
init_sigmasq   = 13.059
init_range_lon = 0.195 
init_range_lat = 0.154 
init_advec_lat = 0.0218
init_range_time = 1.0
init_advec_lon = -0.1689
init_nugget    = 0.247

# Map parameters
init_phi2 = 1.0 / init_range_lon
init_phi1 = init_sigmasq * init_phi2
init_phi3 = (init_range_lon / init_range_lat)**2
init_phi4 = (init_range_lon / init_range_time)**2

# Create Initial Parameters
initial_vals = [np.log(init_phi1), np.log(init_phi2), np.log(init_phi3), 
                np.log(init_phi4), init_advec_lat, init_advec_lon, np.log(init_nugget)]

params_list = [
    torch.tensor([val], requires_grad=True, dtype=DTYPE, device=DEVICE)
    for val in initial_vals
]

# Mean Ozone
OZONE_MEAN = 260.0

# --- 2. EXACT COVARIANCE ---
def get_model_covariance_on_grid(lags_x, lags_y, lags_t, params):
    phi1, phi2, phi3, phi4 = torch.exp(params[0]), torch.exp(params[1]), torch.exp(params[2]), torch.exp(params[3])
    advec_lat, advec_lon = params[4], params[5]
    sigmasq = phi1 / phi2

    u_lat_eff = lags_x - advec_lat * lags_t
    u_lon_eff = lags_y - advec_lon * lags_t
    
    dist_sq = (u_lat_eff.pow(2) * phi3) + (u_lon_eff.pow(2)) + (lags_t.pow(2) * phi4)
    distance = torch.sqrt(dist_sq + 1e-8)
    
    return sigmasq * torch.exp(-distance * phi2)

# --- 3. FFT SIMULATION ---
def generate_exact_gems_field(lat_coords, lon_coords, t_steps, params):
    Nx = len(lat_coords)
    Ny = len(lon_coords)
    Nt = t_steps
    
    print(f"Exact Grid Size: {Nx} (Lat) x {Ny} (Lon) x {Nt} (Time) = {Nx*Ny*Nt} points")
    
    dlat = float(lat_coords[1] - lat_coords[0])
    dlon = float(lon_coords[1] - lon_coords[0])
    dt = 1.0 
    
    Px, Py, Pt = 2*Nx, 2*Ny, 2*Nt
    
    Lx_len = Px * dlat   
    lags_x = torch.arange(Px, device=DEVICE, dtype=DTYPE) * dlat
    lags_x[Px//2:] -= Lx_len 
    
    Ly_len = Py * dlon   
    lags_y = torch.arange(Py, device=DEVICE, dtype=DTYPE) * dlon
    lags_y[Py//2:] -= Ly_len

    Lt_len = Pt * dt     
    lags_t = torch.arange(Pt, device=DEVICE, dtype=DTYPE) * dt
    lags_t[Pt//2:] -= Lt_len

    L_x, L_y, L_t = torch.meshgrid(lags_x, lags_y, lags_t, indexing='ij')
    C_vals = get_model_covariance_on_grid(L_x, L_y, L_t, params)

    S = torch.fft.fftn(C_vals)
    S.real = torch.clamp(S.real, min=0)

    random_phase = torch.fft.fftn(torch.randn(Px, Py, Pt, device=DEVICE, dtype=DTYPE))
    weighted_freq = torch.sqrt(S.real) * random_phase
    field_sim = torch.fft.ifftn(weighted_freq).real
    
    return field_sim[:Nx, :Ny, :Nt]

# --- 4. REGULAR GRID FUNCTIONS (FIXED WITH ROUNDING) ---

def make_target_grid(lat_start, lat_end, lat_step, lon_start, lon_end, lon_step, device, dtype):
    """
    Constructs a grid explicitly from start to end.
    CRITICAL: Includes rounding to 4 decimal places to prevent "Tensor size does not match" errors.
    """
    # 1. Generate Latitudes (Descending from 5.0)
    # We use a small epsilon to ensure the 'end' is included if it's a multiple
    lats = torch.arange(lat_start, lat_end - 0.0001, lat_step, device=device, dtype=dtype)
    lats = torch.round(lats * 10000) / 10000  # <--- FIX: Round to 4 decimals
    
    # 2. Generate Longitudes (Descending from 133.0)
    lons = torch.arange(lon_start, lon_end - 0.0001, lon_step, device=device, dtype=dtype)
    lons = torch.round(lons * 10000) / 10000  # <--- FIX: Round to 4 decimals

    print(f"Grid Generation debug: Lat Range {lats[0]:.4f}-{lats[-1]:.4f}, Lon Range {lons[0]:.4f}-{lons[-1]:.4f}")
    print(f"Unique Lats: {len(lats)}, Unique Lons: {len(lons)}")

    # 3. Meshgrid (indexing='ij' -> Lat is rows, Lon is cols)
    grid_lat, grid_lon = torch.meshgrid(lats, lons, indexing='ij')

    # 4. Flatten
    flat_lats = grid_lat.flatten()
    flat_lons = grid_lon.flatten()

    # 5. Stack
    center_points = torch.stack([flat_lats, flat_lons], dim=1)
    
    # Return grid AND dimensions (Nx, Ny) for verification
    return center_points, len(lats), len(lons)

def coarse_by_center_tensor(input_map_tensors: dict, target_grid_tensor: torch.Tensor):
    coarse_map = {}
    
    # BallTree requires CPU Numpy
    query_points_np = target_grid_tensor.cpu().numpy()
    query_points_rad = np.radians(query_points_np)
    
    for key, val_tensor in input_map_tensors.items():
        # Source locations (Perturbed)
        source_locs_np = val_tensor[:, :2].cpu().numpy()
        source_locs_rad = np.radians(source_locs_np)
        
        # NN Search
        tree = BallTree(source_locs_rad, metric='haversine')
        dist, ind = tree.query(query_points_rad, k=1)
        nearest_indices = ind.flatten()
        
        # Map values back to tensor
        indices_tensor = torch.tensor(nearest_indices, device=val_tensor.device, dtype=torch.long)
        gathered_vals = val_tensor[indices_tensor, 2]
        gathered_times = val_tensor[indices_tensor, 3]
        
        # Construct Regular Tensor
        new_tensor = torch.stack([
            target_grid_tensor[:, 0], # Regular Lat
            target_grid_tensor[:, 1], # Regular Lon
            gathered_vals,            # Mapped Value
            gathered_times            # Mapped Time
        ], dim=1)
        
        coarse_map[key] = new_tensor

    return coarse_map

# --- 5. EXECUTION ---

# Simulation Grid (Base)
lats_sim = torch.arange(0, 5.0 + 0.001, 0.044, device=DEVICE, dtype=DTYPE)
lons_sim = torch.arange(123.0, 133.0 + 0.001, 0.063, device=DEVICE, dtype=DTYPE)
t_def = 8
LOC_ERR_STD = 0.01 

print("1. Generating True Field...")
sim_field = generate_exact_gems_field(lats_sim, lons_sim, t_def, params_list)

print("2. Formatting Output...")
input_map = {}
aggregated_list = [] 
nugget_std = torch.sqrt(torch.exp(params_list[6]))

# Create Base Grid for Perturbation (Lat/Lon Descending)
lats_flip = torch.flip(lats_sim, dims=[0])
lons_flip = torch.flip(lons_sim, dims=[0])
grid_lat, grid_lon = torch.meshgrid(lats_flip, lons_flip, indexing='ij')
flat_lats = grid_lat.flatten()
flat_lons = grid_lon.flatten()

for t in range(t_def):
    field_t = sim_field[:, :, t] 
    field_t_flipped = torch.flip(field_t, dims=[0, 1]) 
    flat_vals = field_t_flipped.flatten()
    
    # Add Nugget Noise
    obs_vals = flat_vals + (torch.randn_like(flat_vals) * nugget_std) + OZONE_MEAN
    
    # Add Location Perturbation
    lat_noise = torch.randn_like(flat_lats) * LOC_ERR_STD
    lon_noise = torch.randn_like(flat_lons) * LOC_ERR_STD
    perturbed_lats = flat_lats + lat_noise
    perturbed_lons = flat_lons + lon_noise

    time_val = 21.0 + t
    flat_times = torch.full_like(flat_lats, time_val)
    
    # Create the Irregular Data Tensor
    row_tensor = torch.stack([perturbed_lats, perturbed_lons, obs_vals, flat_times], dim=1)
    
    key_str = f'2024_07_y24m07day01_hm{t:02d}:53'
    input_map[key_str] = row_tensor.detach()
    aggregated_list.append(input_map[key_str])

aggregated_data = torch.cat(aggregated_list, dim=0)
print(f"Aggregated Tensor Shape (Perturbed): {aggregated_data.shape}")

# --- 6. ENFORCE REGULAR GRID ---
print("\n--- Enforcing Regular Grid ---")

# Step sizes
step_lat = 0.044
step_lon = 0.063

# Generate Target Grid: TOP-DOWN to ensure 5.0 and 133.0 are included
# 5.0 -> 0.0 (Descending) | 133.0 -> 123.0 (Descending)
target_grid, Nx_reg, Ny_reg = make_target_grid(
    lat_start=5.0, lat_end=0.0, lat_step=-step_lat,
    lon_start=133.0, lon_end=123.0, lon_step=-step_lon,
    device=DEVICE, dtype=DTYPE
)

print(f"Target Regular Grid Shape: {target_grid.shape}")

# Map Perturbed Data to Regular Grid
coarse_map = coarse_by_center_tensor(input_map, target_grid)

# Aggregate Clean Data
coarse_aggregated_list = list(coarse_map.values())
coarse_aggregated_data = torch.cat(coarse_aggregated_list, dim=0)

# --- CRITICAL: OVERWRITE VARIABLES FOR DOWNSTREAM TASKS ---
input_map = coarse_map
aggregated_data = coarse_aggregated_data

print("\n--- Regularization Complete ---")
print(f"Final Data Shape: {aggregated_data.shape}")

# --- 7. VERIFICATION ---
# Verify integrity for Whittle (Must be perfect rect)
u_lat = torch.unique(aggregated_data[:, 0])
u_lon = torch.unique(aggregated_data[:, 1])

print(f"\nGrid Integrity Check:")
print(f"Unique Latitudes: {len(u_lat)}")
print(f"Unique Longitudes: {len(u_lon)}")
print(f"Expected Total Points per Day: {len(u_lat) * len(u_lon)}")
print(f"Actual Points per Day: {aggregated_data.shape[0] // t_def}")

if (len(u_lat) * len(u_lon)) == (aggregated_data.shape[0] // t_def):
    print("SUCCESS: Grid is perfect and ready for Whittle.")
else:
    print("WARNING: Grid mismatch detected! Whittle will fail.")

Simulating on: cpu
1. Generating True Field...
Exact Grid Size: 114 (Lat) x 159 (Lon) x 8 (Time) = 145008 points
2. Formatting Output...
Aggregated Tensor Shape (Perturbed): torch.Size([145008, 4])

--- Enforcing Regular Grid ---
Grid Generation debug: Lat Range 5.0000-0.0280, Lon Range 133.0000-123.0460
Unique Lats: 114, Unique Lons: 159
Target Regular Grid Shape: torch.Size([18126, 2])

--- Regularization Complete ---
Final Data Shape: torch.Size([145008, 4])

Grid Integrity Check:
Unique Latitudes: 114
Unique Longitudes: 159
Expected Total Points per Day: 18126
Actual Points per Day: 18126
SUCCESS: Grid is perfect and ready for Whittle.


set up

In [20]:
from GEMS_TCO import orderings as _orderings

inputmap = coarse_map
aggregated_data = coarse_aggregated_data

def get_spatial_ordering(
        
        input_maps: torch.Tensor,
        mm_cond_number: int = 10
    ) -> Tuple[np.ndarray, np.ndarray]:
        
        key_list = list(input_maps.keys())
        data_for_coord = input_maps[key_list[0]]
        
        # --- FIX START ---
        # Check if input is Tensor, if so convert to Numpy for KDTree processing
        if isinstance(data_for_coord, torch.Tensor):
            data_for_coord = data_for_coord.cpu().numpy()
        # --- FIX END ---

        x1 = data_for_coord[:, 0]
        y1 = data_for_coord[:, 1]
        
        # Now this works because x1, y1 are numpy arrays
        coords1 = np.stack((x1, y1), axis=-1)

        # Calculate MaxMin ordering
        ord_mm = _orderings.maxmin_cpp(coords1)
        
        # Reorder coordinates to find nearest neighbors
        data_for_coord_reordered = data_for_coord[ord_mm]
        coords1_reordered = np.stack(
            (data_for_coord_reordered[:, 0], data_for_coord_reordered[:, 1]), 
            axis=-1
        )
        
        # Calculate nearest neighbors map
        nns_map = _orderings.find_nns_l2(locs=coords1_reordered, max_nn=mm_cond_number)
        
        return ord_mm, nns_map

ord_mm, nns_map = get_spatial_ordering(input_map, mm_cond_number=10)

In [21]:
mm_input_map = {}
for key in input_map:
    mm_input_map[key] = input_map[key][ord_mm]  # Extract only Lat and Lon columns

# Fit vecchia max min time 2 

In [24]:
import torch
import numpy as np
import time

# --- CONFIGURATION ---
v = 0.5              # Smoothness
mm_cond_number = 8   # Neighbors
nheads = 0           # 0 = Pure Vecchia
lr = 1.0             # LBFGS learning rate
LBFGS_MAX_STEPS = 10
LBFGS_HISTORY_SIZE = 100
LBFGS_LR = 1.0
LBFGS_MAX_EVAL = 100    

#DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

# --- 1. SETUP PARAMETERS (List of Scalars) ---
# Truth: [4.18, 1.94, 0.24, -3.97, 0.014, -0.20, -0.85]
init_sigmasq   = 13.059
init_range_lat = 0.154 
init_range_lon = 0.195
init_advec_lat = 0.0218
init_range_time = 1.0
init_advec_lon = -0.1689
init_nugget    = 0.247

# Map model parameters to the 'phi' reparameterization
init_phi2 = 1.0 / init_range_lon                # 1/range_lon
init_phi1 = init_sigmasq * init_phi2            # sigmasq / range_lon
init_phi3 = (init_range_lon / init_range_lat)**2  # (range_lon / range_lat)^2
init_phi4 = (init_range_lon / init_range_time)**2      # (range_lon / range_time)^2

# Create Initial Parameters (Float64, Requires Grad)
initial_vals = [np.log(init_phi1), np.log(init_phi2), np.log(init_phi3), 
                np.log(init_phi4), init_advec_lat, init_advec_lon, np.log(init_nugget)]

params_list = [
    torch.tensor([val], requires_grad=True, dtype=torch.float64, device=DEVICE)
    for val in initial_vals
]

# --- 2. INSTANTIATE MODEL ---
print(f'\n{"="*40}')
print(f'--- Initializing VecchiaBatched Model ---')
print(f'{"="*40}')

if isinstance(aggregated_data, torch.Tensor):
    aggregated_data = aggregated_data.to(DEVICE)

# Instantiate
model_instance = kernels_reparam_space_time_gpu.fit_vecchia_lbfgs(
    smooth=v,
    input_map=mm_input_map,
    aggregated_data=aggregated_data,
    nns_map=nns_map,
    mm_cond_number=mm_cond_number,
    nheads=nheads
)



# --- 3. OPTIMIZATION LOOP ---
print(f'\n{"="*40}')
print(f'--- Running L-BFGS Optimization ---')
print(f'{"="*40}')

# Optimizer takes the LIST of scalars
optimizer_vecc = model_instance.set_optimizer(
            params_list,     
            lr=LBFGS_LR,            
            max_iter=LBFGS_MAX_EVAL,        
            history_size=LBFGS_HISTORY_SIZE 
        )

start_time = time.time()

out, steps_ran = model_instance.fit_vecc_lbfgs(
        params_list,
        optimizer_vecc,
        # covariance_function argument is GONE
        max_steps=LBFGS_MAX_STEPS, 
        grad_tol=1e-7
    )


end_time = time.time()
epoch_time = end_time - start_time

print(f"\nOptimization finished in {epoch_time:.2f}s.")
print(f"Results after {steps_ran} steps: {out}")
print(f"Final Params: {torch.cat(params_list).detach().cpu().numpy()}")

Using device: cpu

--- Initializing VecchiaBatched Model ---

--- Running L-BFGS Optimization ---
Pre-computing Batched Tensors (Padding Strategy)... Done. Heads: 0, Batched Tails: 145008
--- Starting Batched L-BFGS Optimization (GPU) ---
--- Step 1/10 / Loss: 1.260370 ---
  Param 0: Value=4.3720, Grad=-6.014047242982236e-06
  Param 1: Value=1.9128, Grad=1.6234249897639368e-06
  Param 2: Value=0.1192, Grad=-7.454680619066402e-07
  Param 3: Value=-3.6115, Grad=-6.963195919033935e-07
  Param 4: Value=0.0179, Grad=-2.572804173317511e-07
  Param 5: Value=-0.1594, Grad=3.272672421006373e-06
  Param 6: Value=-1.7218, Grad=-3.587402327247341e-08
  Max Abs Grad: 6.014047e-06
------------------------------
--- Step 2/10 / Loss: 1.256587 ---
  Param 0: Value=4.3720, Grad=-6.014047242982236e-06
  Param 1: Value=1.9128, Grad=1.6234249897639368e-06
  Param 2: Value=0.1192, Grad=-7.454680619066402e-07
  Param 3: Value=-3.6115, Grad=-6.963195919033935e-07
  Param 4: Value=0.0179, Grad=-2.572804173317

# fit dw

difference data

In [22]:
a = [11.0474, 0.0623, 0.2445, 1.0972, 0.0101, -0.1671, 1.1825]
day = 0 # 0 index
lat_range= [0,5]
lon_range= [123.0, 133.0]
#lat_range= [1,3]
#lon_range= [125, 129.0]

daily_aggregated_tensors_dw = [aggregated_data]
daily_hourly_maps_dw = [input_map]

db = debiased_whittle.debiased_whittle_preprocess(daily_aggregated_tensors_dw, daily_hourly_maps_dw, day_idx=day, params_list=a, lat_range=lat_range, lon_range=lon_range)


subsetted_aggregated_day = db.generate_spatially_filtered_days(0,5,123,133)
print(subsetted_aggregated_day.shape)
N2= subsetted_aggregated_day.shape[0]
print(N2)
subsetted_aggregated_day[:20]

torch.Size([142832, 4])
142832


tensor([[ 2.8000e-02,  1.2305e+02,  0.0000e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2311e+02, -3.9546e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2317e+02,  1.4146e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2323e+02, -1.0373e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2330e+02, -3.1927e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2336e+02, -5.3402e-01,  2.1000e+01],
        [ 2.8000e-02,  1.2342e+02,  3.0879e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2349e+02, -9.5345e-01,  2.1000e+01],
        [ 2.8000e-02,  1.2355e+02,  5.9970e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2361e+02, -2.3190e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2368e+02, -2.5883e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2374e+02,  4.1502e-01,  2.1000e+01],
        [ 2.8000e-02,  1.2380e+02, -3.2147e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2386e+02, -6.4375e-01,  2.1000e+01],
        [ 2.8000e-02,  1.2393e+02,  1.7592e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2399e+02,  3.2374e+00,  2.1000e+01],
        

In [23]:

dwl = debiased_whittle.debiased_whittle_likelihood()
if __name__ == '__main__':
    start_time = time.time()

    # --- Configuration ---
    DAY_TO_RUN = 3 # data is decided above
    TAPERING_FUNC = dwl.cgn_hamming # Use Hamming taper
    NUM_RUNS = 1
    MAX_STEPS = 20 # L-BFGS usually converges in far fewer steps
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {DEVICE}")

    DELTA_LAT, DELTA_LON = 0.044, 0.063 

    LAT_COL, LON_COL = 0, 1
    VAL_COL = 2 # Spatially differenced value
    TIME_COL = 3


    cur_df =subsetted_aggregated_day
    
    if cur_df.numel() == 0 or cur_df.shape[1] <= max(LAT_COL, LON_COL, VAL_COL, TIME_COL):
        print(f"Error: Data for Day {DAY_TO_RUN} is empty or invalid.")
        exit()

    unique_times = torch.unique(cur_df[:, TIME_COL])
    time_slices_list = [cur_df[cur_df[:, TIME_COL] == t_val] for t_val in unique_times]

    # --- 1. Pre-compute J-vector, Taper Grid, and Taper Autocorrelation ---
    print("Pre-computing J-vector (Hamming taper)...")
    
    # --- 💥 REVISED: Renamed 'p' to 'p_time' 💥 ---
    J_vec, n1, n2, p_time, taper_grid = dwl.generate_Jvector_tapered( 
        time_slices_list,
        tapering_func=TAPERING_FUNC, 
        lat_col=LAT_COL, lon_col=LON_COL, val_col=VAL_COL,
        device=DEVICE
    )

    if J_vec is None or J_vec.numel() == 0 or n1 == 0 or n2 == 0 or p_time == 0:
       print(f"Error: J-vector generation failed for Day {DAY_TO_RUN}.")
       exit()
       
    print("Pre-computing sample periodogram...")
    I_sample = dwl.calculate_sample_periodogram_vectorized(J_vec)

    print("Pre-computing Hamming taper autocorrelation...")
    taper_autocorr_grid = dwl.calculate_taper_autocorrelation_fft(taper_grid, n1, n2, DEVICE)

    if torch.isnan(I_sample).any() or torch.isinf(I_sample).any():
        print("Error: NaN/Inf in sample periodogram.")
        exit()
    if torch.isnan(taper_autocorr_grid).any() or torch.isinf(taper_autocorr_grid).any():
        print("Error: NaN/Inf in taper autocorrelation.")
        exit()

    print(f"Data grid: {n1}x{n2}, {p_time} time points. J-vector, Periodogram, Taper Autocorr on {DEVICE}.")
    # --- END REVISION ---

    # --- 2. Optimization Loop ---
    all_final_results = []
    all_final_losses = []

    for i in range(NUM_RUNS):
        print(f"\n{'='*30} Initialization Run {i+1}/{NUM_RUNS} {'='*30}")

        # --- 7-PARAMETER initialization ---
        ''' 
        init_sigmasq   = 15.0
        init_range_lat = 0.66 
        init_range_lon = 0.7 
        init_nugget    = 1.5
        init_beta      = 0.1  # Temporal range ratio
        init_advec_lat = 0.02
        init_advec_lon = -0.08
        '''
        init_sigmasq   = 13.059
        init_range_lat = 0.154 
        init_range_lon = 0.195
        init_advec_lat = 0.0218
        init_range_time = 0.7
        init_advec_lon = -0.1689
        init_nugget    = 0.247

        init_phi2 = 1.0 / init_range_lon
        init_phi1 = init_sigmasq * init_phi2
        init_phi3 = (init_range_lon / init_range_lat)**2
        # Change needed to match the spatial-temporal distance formula:
        init_phi4 = (init_range_lon / init_range_time)**2      # (range_lon / range_time)^2

        initial_params_values = [
            np.log(init_phi1),    # [0] log_phi1
            np.log(init_phi2),    # [1] log_phi2
            np.log(init_phi3),    # [2] log_phi3
            np.log(init_phi4),    # [3] log_phi4
            init_advec_lat,       # [4] advec_lat (NOT log)
            init_advec_lon,       # [5] advec_lon (NOT log)
            np.log(init_nugget)   # [6] log_nugget
        ]
        
        print(f"Starting with FIXED params (raw log-scale): {[round(p, 4) for p in initial_params_values]}")

        params_list = [
            Parameter(torch.tensor([val], dtype=torch.float64))
            for val in initial_params_values
        ]

        # Helper to define the boundary globally for clarity
        NUGGET_LOWER_BOUND = 0.05
        LOG_NUGGET_LOWER_BOUND = np.log(NUGGET_LOWER_BOUND) # Approx -2.9957

        # --- 💥 REVISED: Use L-BFGS Optimizer 💥 ---
        optimizer = torch.optim.LBFGS(
            params_list,
            lr=1.0,           # Initial step length for line search
            max_iter=20,      # Iterations per step
            history_size=100,
            line_search_fn="strong_wolfe", # Often more robust
            tolerance_grad=1e-5
        )
        # --- END REVISION ---

        print(f"Starting optimization run {i+1} on device {DEVICE} (Hamming, 7-param ST kernel, L-BFGS)...")
        
        # --- 💥 REVISED: Call L-BFGS trainer, pass p_time 💥 ---
        nat_params_str, phi_params_str, raw_params_str, loss, steps_run = dwl.run_lbfgs_tapered(
            params_list=params_list,
            optimizer=optimizer,
            I_sample=I_sample,
            n1=n1, n2=n2, p_time=p_time,
            taper_autocorr_grid=taper_autocorr_grid, 
            max_steps=MAX_STEPS,
            device=DEVICE
        )
        # --- END REVISION ---
        
        if loss is not None:
            all_final_results.append((nat_params_str, phi_params_str, raw_params_str))
            all_final_losses.append(loss)
        else:
            all_final_losses.append(float('inf'))

    print(f"\n\n{'='*25} Overall Result from Run {'='*25} {'='*25}")
    
    valid_losses = [l for l in all_final_losses if l is not None and l != float('inf')]

    if not valid_losses:
        print(f"The run failed or resulted in an invalid loss for Day {DAY_TO_RUN}.")
    else:
        best_loss = min(valid_losses)
        best_run_index = all_final_losses.index(best_loss)
        best_results = all_final_results[best_run_index]
        
        print(f"Best Run Loss: {best_loss} (after {steps_run} steps)")
        print(f"Final Parameters (Natural Scale): {best_results[0]}")
        print(f"Final Parameters (Phi Scale)    : {best_results[1]}")
        print(f"Final Parameters (Raw Log Scale): {best_results[2]}")

    end_time = time.time()
    print(f"\nTotal execution time: {end_time - start_time:.2f} seconds")

Using device: cpu
Pre-computing J-vector (Hamming taper)...
Pre-computing sample periodogram...
Pre-computing Hamming taper autocorrelation...
Data grid: 113x158, 8 time points. J-vector, Periodogram, Taper Autocorr on cpu.

============================== Initialization Run 1/1 ==============================
Starting with FIXED params (raw log-scale): [4.2042, 1.6348, 0.4721, -2.5562, 0.0218, -0.1689, -1.3984]
Starting optimization run 1 on device cpu (Hamming, 7-param ST kernel, L-BFGS)...
--- Step 1/20 ---
 Loss: 1.839209 | Max Grad: 4.004900e-05
  Params (Raw Log): log_phi1: 4.2999, log_phi2: 1.7757, log_phi3: 0.1161, log_phi4: -3.2281, advec_lat: 0.0202, advec_lon: -0.1415, log_nugget: -1.0194
--- Step 2/20 ---
 Loss: 1.773573 | Max Grad: 5.320679e-06
  Params (Raw Log): log_phi1: 4.3000, log_phi2: 1.7759, log_phi3: 0.1161, log_phi4: -3.2281, advec_lat: 0.0202, advec_lon: -0.1415, log_nugget: -1.0197
--- Step 3/20 ---
 Loss: 1.773573 | Max Grad: 5.320679e-06
  Params (Raw Log): log